In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Here, the dataset contains articles(excerpt column) which are evaluated based on the level of difficulty of the articles and shown in the target column which we need to predict for the test.csv file as well as the submission.csv file 

In [ ]:
'''
Reference:
1. https://www.kaggle.com/pawan2905/commonlit-readability-prize
2. https://www.kaggle.com/ruchi798/commonlit-readability-prize-eda-baseline
3. https://www.kaggle.com/ibtesama/getting-started-with-a-movie-recommendation-system

'''
import numpy as np
import pandas as pd

import os

import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# sklearn 
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error as mse


train = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
test = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
sample = pd.read_csv('/kaggle/input/commonlitreadabilityprize/sample_submission.csv')


From reference 3, we will learn how to use the TfidfVectorizer to convert the words into the features as shown below:

In [ ]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df2['overview'] = df2['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df2['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape
#Out[12]:
#(4803, 20978)

We see that over 20,000 different words were used to describe the 4800 movies in their dataset from the value of the tfidf_matrix.shape. Now,we continue with our code below.

In [ ]:
#print(train)

print(train.head(20))

In [ ]:
print('df.shape .......', train.shape)

In [ ]:
# Removing unnecessary columns
#train.drop(['url_legal', 'license'], axis=1, inplace=True)

In [ ]:

# text preprocessing helper functions

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text



In [ ]:

def text_preprocessing(text):
    """
    Cleaning and parsing the text.

    """
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    nopunc = clean_text(text)
    tokenized_text = tokenizer.tokenize(nopunc)
    #remove_stopwords = [w for w in tokenized_text if w not in stopwords.words('english')]
    combined_text = ' '.join(tokenized_text)
    return combined_text

In [ ]:
print(train['excerpt'])

In [ ]:
train['excerpt_clean'] = train['excerpt'].apply(str).apply(lambda x: text_preprocessing(x))
print(train['excerpt_clean'])

In [ ]:
test['excerpt_clean'] =test['excerpt'].apply(str).apply(lambda x: text_preprocessing(x))
train['excerpt_len'] = train['excerpt_clean'].astype(str).apply(len)
train['excerpt_count'] = train['excerpt_clean'].apply(lambda x: len(str(x).split()))

In [ ]:
print(train['excerpt_len'], train['excerpt_count'])

In [ ]:
vectorizer = TfidfVectorizer(analyzer='char',ngram_range=(1,6))
X = vectorizer.fit_transform(train.excerpt_clean)
X.shape
print('X.shape .......', X.shape)

In [ ]:
lr = LinearRegression()

lr.fit(X, train.target)


In [ ]:
sub = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv',index_col='id')
print(sub)
x = vectorizer.transform(sub.excerpt)

In [ ]:
p = lr.predict(x)
sub['target'] = p
sub[['target']].to_csv('submission.csv')